In [1]:
import pandas as pd

In [2]:
# Gold
df = pd.read_csv("Gold.csv",sep = ",")
df['Date Local'] = pd.to_datetime(df['Date Local'], errors='coerce')
df_jan = df[(df['Date Local'] >= '2019-01-01') & (df['Date Local'] <= '2019-01-31')]
df_jan = df_jan[['Latitude', 'Longitude', 'AQI']]
# df_jan = df_jan[['Latitude', 'Longitude', 'Arithmetic Mean']]
df_avg = df_jan.groupby(['Latitude', 'Longitude'], as_index=False).mean()
df_avg.to_csv("Gold_clean.csv", index=False)

/var/folders/9n/7bxqt9q92tl6lchpfwnymvjh0000gn/T/ipykernel_76533/1275416242.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Gold.csv",sep = ",")


In [3]:
#CMAQ
df3 = pd.read_csv("CMAQ.csv", skiprows=2)
df3 = df3.iloc[1:]
df4 = df3[['latitude', 'longitude', 'PM25_AVG']].copy()
df4['latitude'] = pd.to_numeric(df4['latitude'], errors='coerce').round(6)
df4['longitude'] = pd.to_numeric(df4['longitude'], errors='coerce').round(6)
df4['PM25_AVG'] = pd.to_numeric(df4['PM25_AVG'], errors='coerce')
df_avg2 = df4.groupby(['latitude', 'longitude'], as_index=False).mean()
df_avg2.to_csv("CMAQ_clean.csv", index=False)

/var/folders/9n/7bxqt9q92tl6lchpfwnymvjh0000gn/T/ipykernel_5704/3912202928.py:2: DtypeWarning: Columns (2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("CMAQ.csv", skiprows=2)


In [2]:
import pandas as pd

def print_range(name, df):
    print(f"{name}: lat=({df['lat'].min():.3f}, {df['lat'].max():.3f}), "
          f"lon=({df['lon'].min():.3f}, {df['lon'].max():.3f}), "
          f"count={len(df)}")

# =============================
# Full-land datasets
# =============================
df_g = pd.read_csv("Gold_clean_land.csv")
df_c = pd.read_csv("CMAQ_clean_land.csv")

print_range("Gold_clean_land", df_g)
print_range("CMAQ_clean_land", df_c)

# =============================
# Subsets: CA / NE / SE
# =============================
regions = ["CA", "NE", "SE"]

for r in regions:
    df_frm = pd.read_csv(f"FRM_{r}_subset.csv")
    df_cmaq = pd.read_csv(f"CMAQ_{r}_subset.csv")
    
    print("\n--- Region:", r, "---")
    print_range(f"FRM_{r}_subset", df_frm)
    print_range(f"CMAQ_{r}_subset", df_cmaq)


Gold_clean_land: lat=(25.472, 48.760), lon=(-124.179, -68.016), count=918
CMAQ_clean_land: lat=(25.166, 49.318), lon=(-124.639, -67.110), count=53757

--- Region: CA ---
FRM_CA_subset: lat=(32.631, 41.727), lon=(-124.179, -115.483), count=112
CMAQ_CA_subset: lat=(32.554, 42.009), lon=(-124.320, -114.181), count=2787

--- Region: NE ---
FRM_NE_subset: lat=(39.422, 44.393), lon=(-80.485, -70.971), count=96
CMAQ_NE_subset: lat=(38.959, 44.999), lon=(-80.520, -69.975), count=1953

--- Region: SE ---
FRM_SE_subset: lat=(25.472, 34.979), lon=(-88.088, -79.767), count=78
CMAQ_SE_subset: lat=(25.166, 35.178), lon=(-88.458, -78.691), count=3573


In [3]:
# Get CVS1: 
# This is U2 Part (about 1000 data) si, Z1 and closest uj and Z2
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load and rename
df_gold = pd.read_csv("Gold_clean_land.csv")
df_cmaq = pd.read_csv("CMAQ_clean_land.csv")

df_gold = df_gold.rename(columns={'lat': 'si_lat', 'lon': 'si_lon', 'AQI': 'Z1'})
#df_gold = df_gold.rename(columns={'Latitude': 'si_lat', 'Longitude': 'si_lon', 'Arithmetic Mean': 'Z1'})

df_cmaq = df_cmaq.rename(columns={'lat': 'uj_lat', 'lon': 'uj_lon', 'PM25': 'Z2'})

# Round coordinates to avoid float precision issues
df_gold['si_lat'] = df_gold['si_lat'].round(5)
df_gold['si_lon'] = df_gold['si_lon'].round(5)
df_cmaq['uj_lat'] = df_cmaq['uj_lat'].round(5)
df_cmaq['uj_lon'] = df_cmaq['uj_lon'].round(5)

# Prepare for NearestNeighbor
X_gold = df_gold[['si_lat', 'si_lon']].to_numpy()
X_cmaq = df_cmaq[['uj_lat', 'uj_lon']].to_numpy()

# Find nearest uj for each si
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nbrs.fit(X_cmaq)
distances, indices = nbrs.kneighbors(X_gold)

# Get nearest Z2
nearest_cmaq = df_cmaq.iloc[indices.flatten()].reset_index(drop=True)
df_gold_reset = df_gold.reset_index(drop=True)

# Merge and round again just in case
df_matched = pd.concat([df_gold_reset, nearest_cmaq], axis=1)
df_matched[['uj_lat', 'uj_lon']] = df_matched[['uj_lat', 'uj_lon']].round(5)

# Final output
df_csv1 = df_matched[['si_lat', 'si_lon', 'uj_lat', 'uj_lon', 'Z1', 'Z2']]
df_csv1.to_csv("CSV1_land.csv", index=False)

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

regions = ["CA", "NE", "SE"]

for reg in regions:
    print(f"Processing region: {reg}...")

    # Load subset files
    df_gold = pd.read_csv(f"FRM_{reg}_subset.csv")
    df_cmaq = pd.read_csv(f"CMAQ_{reg}_subset.csv")

    # Rename columns
    df_gold = df_gold.rename(columns={'lat': 'si_lat', 'lon': 'si_lon', 'AQI': 'Z1'})
    df_cmaq = df_cmaq.rename(columns={'lat': 'uj_lat', 'lon': 'uj_lon', 'PM25': 'Z2'})

    # Round coords
    df_gold[['si_lat', 'si_lon']] = df_gold[['si_lat', 'si_lon']].round(5)
    df_cmaq[['uj_lat', 'uj_lon']] = df_cmaq[['uj_lat', 'uj_lon']].round(5)

    # Prepare nearest neighbor search
    X_gold = df_gold[['si_lat', 'si_lon']].to_numpy()
    X_cmaq = df_cmaq[['uj_lat', 'uj_lon']].to_numpy()

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
    nbrs.fit(X_cmaq)
    distances, indices = nbrs.kneighbors(X_gold)

    # Match nearest CMAQ point
    nearest_cmaq = df_cmaq.iloc[indices.flatten()].reset_index(drop=True)
    df_gold_reset = df_gold.reset_index(drop=True)

    # Merge
    df_matched = pd.concat([df_gold_reset, nearest_cmaq], axis=1)
    df_matched[['uj_lat', 'uj_lon']] = df_matched[['uj_lat', 'uj_lon']].round(5)

    # Final CSV1 output for region
    df_csv1 = df_matched[['si_lat', 'si_lon', 'uj_lat', 'uj_lon', 'Z1', 'Z2']]
    df_csv1.to_csv(f"CSV1_{reg}.csv", index=False)

    print(f"Saved: CSV1_{reg}.csv  (N={len(df_csv1)})\n")


Processing region: CA...
Saved: CSV1_CA.csv  (N=108)

Processing region: NE...
Saved: CSV1_NE.csv  (N=95)

Processing region: SE...
Saved: CSV1_SE.csv  (N=77)



In [5]:
# U1/U2: CSV2
df_cmaq = pd.read_csv("CMAQ_clean.csv")
df_cmaq = df_cmaq.rename(columns={'latitude': 'uj_lat', 'longitude': 'uj_lon', 'PM25_AVG': 'Z2'})
df_csv1 = pd.read_csv("CSV1.csv")

df_cmaq['uj_lat'] = df_cmaq['uj_lat'].round(5)
df_cmaq['uj_lon'] = df_cmaq['uj_lon'].round(5)
df_csv1['uj_lat'] = df_csv1['uj_lat'].round(5)
df_csv1['uj_lon'] = df_csv1['uj_lon'].round(5)

df_cmaq_unique = df_cmaq.drop_duplicates(subset=['uj_lat', 'uj_lon'])
df_csv1_unique = df_csv1[['uj_lat', 'uj_lon']].drop_duplicates()

df_unmatched = df_cmaq_unique.merge(
    df_csv1_unique,
    on=['uj_lat', 'uj_lon'],
    how='left',
    indicator=True
)

df_unmatched = df_unmatched[df_unmatched['_merge'] == 'left_only'].drop(columns=['_merge'])

df_unmatched.to_csv("CSV2.csv", index=False)



In [3]:
import pandas as pd

regions = ["CA", "NE", "SE"]

for reg in regions:
    print(f"Processing region: {reg}...")

    # Load subset files
    df_gold = pd.read_csv(f"FRM_{reg}_subset.csv")
    df_cmaq = pd.read_csv(f"CMAQ_{reg}_subset.csv")

    # Rename columns
    df_gold = df_gold.rename(columns={'lat': 'si_lat', 'lon': 'si_lon'})
    df_cmaq = df_cmaq.rename(columns={'lat': 'uj_lat', 'lon': 'uj_lon', 'PM25': 'Z2'})

    # Round coordinates
    df_gold[['si_lat', 'si_lon']] = df_gold[['si_lat', 'si_lon']].round(5)
    df_cmaq[['uj_lat', 'uj_lon']] = df_cmaq[['uj_lat', 'uj_lon']].round(5)

    # Remove duplicates in CMAQ grid
    df_cmaq_unique = df_cmaq.drop_duplicates(subset=['uj_lat', 'uj_lon'])

    # Only keep coordinates columns for matching
    df_csv1 = pd.read_csv(f"CSV1_{reg}.csv")
    df_csv1_coords = df_csv1[['uj_lat', 'uj_lon']].drop_duplicates()

    # Find CMAQ points **NOT** used in CSV1 (i.e., not matched with FRM)
    df_unmatched = df_cmaq_unique.merge(
        df_csv1_coords,
        on=['uj_lat', 'uj_lon'],
        how='left',
        indicator=True
    )
    df_unmatched = df_unmatched[df_unmatched['_merge'] == 'left_only']
    df_unmatched = df_unmatched.drop(columns=['_merge'])

    # Only keep required columns Z2 + coords
    df_csv2 = df_unmatched[['uj_lat', 'uj_lon', 'Z2']]
    df_csv2.to_csv(f"CSV2_{reg}.csv", index=False)

    print(f"Saved: CSV2_{reg}.csv (N={len(df_csv2)})\n")

print("🎯 Successfully generated CSV2_CA/NE/SE.csv for all regions!")


Processing region: CA...
Saved: CSV2_CA.csv (N=2686)

Processing region: NE...
Saved: CSV2_NE.csv (N=1873)

Processing region: SE...
Saved: CSV2_SE.csv (N=3499)

🎯 Successfully generated CSV2_CA/NE/SE.csv for all regions!


In [4]:
len(df_csv1) 

77

In [15]:
# Train/test split

from sklearn.model_selection import train_test_split
df_csv1_train, df_csv1_test = train_test_split(df_csv1, test_size=0.2, random_state=42)
df_csv1_train.to_csv("CSV1_train.csv", index=False)
df_csv1_test.to_csv("CSV1_test.csv", index=False)

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

regions = ["CA", "NE", "SE"]

for reg in regions:
    print(f"Splitting CSV1 for region: {reg}...")

    df_csv1 = pd.read_csv(f"CSV1_{reg}.csv")

    # 80% training, 20% testing
    df_train, df_test = train_test_split(
        df_csv1, 
        test_size=0.2, 
        random_state=42
    )

    df_train.to_csv(f"CSV1_{reg}_train.csv", index=False)
    df_test.to_csv(f"CSV1_{reg}_test.csv", index=False)

    print(f"Saved: CSV1_{reg}_train.csv (N={len(df_train)})")
    print(f"Saved: CSV1_{reg}_test.csv (N={len(df_test)})\n")

print("🎯 Region-wise train/test split completed successfully!")


Splitting CSV1 for region: CA...
Saved: CSV1_CA_train.csv (N=86)
Saved: CSV1_CA_test.csv (N=22)

Splitting CSV1 for region: NE...
Saved: CSV1_NE_train.csv (N=76)
Saved: CSV1_NE_test.csv (N=19)

Splitting CSV1 for region: SE...
Saved: CSV1_SE_train.csv (N=61)
Saved: CSV1_SE_test.csv (N=16)

🎯 Region-wise train/test split completed successfully!


In [2]:
df_csv1_test = pd.read_csv("CSV1_test.csv")

In [3]:
len(df_csv1_test)

97

In [8]:
import pandas as pd
import numpy as np

df_proj = pd.read_csv("projection_matrix_total.csv")
df_csv1_test = pd.read_csv("CSV1_test.csv")


df_proj_csv1_like = df_proj[(df_proj["Z1"] - df_proj["proj_Z1"]).abs() > 1e-8]


keys = ['uj_lat','uj_lon','Z1','Z2']
def rounded_view(df, cols=keys, nd=6):
    v = df[cols].astype(float).copy()
    for c in cols:
        v[c] = v[c].round(nd)
    return v

rv_proj_csv1_like = rounded_view(df_proj_csv1_like)         
rv_csv1_test      = rounded_view(df_csv1_test)               

# test = CSV1-like ∩ CSV1_test
df_proj_test_idx = rv_proj_csv1_like.merge(
    rv_csv1_test.drop_duplicates(), on=keys, how='inner'
).index


df_proj_test = df_proj_csv1_like.iloc[df_proj_test_idx].copy()


rv_proj_all = rounded_view(df_proj)                          
rv_test_keys = rounded_view(df_proj_test).drop_duplicates()  

anti = rv_proj_all.merge(rv_test_keys, on=keys, how='left', indicator=True)
keep_mask = anti['_merge'].eq('left_only').to_numpy()


df_proj_train = df_proj.loc[keep_mask].copy()


assert len(df_proj) == len(df_proj_train) + len(df_proj_test), \
    f"mismatch: total={len(df_proj)}, train={len(df_proj_train)}, test={len(df_proj_test)}"


df_proj_train.to_csv("projection_matrix_train.csv", index=False)
df_proj_test.to_csv("projection_matrix_test.csv", index=False)


In [6]:
import pandas as pd
import numpy as np

regions = ["CA", "NE", "SE"]

for reg in regions:
    print(f"\n==============================")
    print(f"Processing region: {reg}")
    print("==============================")

    # Load projection matrix & test CSV1 for this region
    df_proj = pd.read_csv(f"projection_matrix_{reg}.csv")
    df_csv1_test = pd.read_csv(f"CSV1_{reg}_test.csv")

    keys = ['uj_lat','uj_lon','Z1','Z2']

    def rounded_view(df, cols=keys, nd=6):
        v = df[cols].astype(float).copy()
        for c in cols:
            v[c] = v[c].round(nd)
        return v

    # Round views for matching
    rv_proj = rounded_view(df_proj)
    rv_csv1_test = rounded_view(df_csv1_test)

    df_proj_test_idx = rv_proj.merge(
        rv_csv1_test.drop_duplicates(),
        on=keys,
        how='inner'
    ).index

    df_proj_test = df_proj.iloc[df_proj_test_idx].copy()

    rv_test_keys = rounded_view(df_proj_test).drop_duplicates()
    anti = rv_proj.merge(rv_test_keys, on=keys, how='left', indicator=True)
    keep_mask = anti['_merge'].eq('left_only').to_numpy()
    df_proj_train = df_proj[keep_mask].copy()

    # Check consistency
    assert len(df_proj) == len(df_proj_train) + len(df_proj_test), \
        f"[{reg}] mismatch: total={len(df_proj)}, train={len(df_proj_train)}, test={len(df_proj_test)}"

    # Save output
    df_proj_train.to_csv(f"projection_matrix_{reg}_train.csv", index=False)
    df_proj_test.to_csv(f"projection_matrix_{reg}_test.csv", index=False)

    print(f"✔ Saved projection_matrix_{reg}_train.csv (N={len(df_proj_train)})")
    print(f"✔ Saved projection_matrix_{reg}_test.csv  (N={len(df_proj_test)})")

print("\n🎯 Projection matrix train/test split done for all regions!")



Processing region: CA
✔ Saved projection_matrix_CA_train.csv (N=2773)
✔ Saved projection_matrix_CA_test.csv  (N=22)

Processing region: NE
✔ Saved projection_matrix_NE_train.csv (N=1950)
✔ Saved projection_matrix_NE_test.csv  (N=19)

Processing region: SE
✔ Saved projection_matrix_SE_train.csv (N=3560)
✔ Saved projection_matrix_SE_test.csv  (N=16)

🎯 Projection matrix train/test split done for all regions!


In [7]:
import pandas as pd
import numpy as np

regions = ["CA", "NE", "SE"]

for reg in regions:
    print(f"\n==============================")
    print(f"Processing region: {reg}")
    print("==============================")

    df_proj = pd.read_csv(f"projection_matrix_{reg}.csv")
    df_csv1_test = pd.read_csv(f"CSV1_{reg}_test.csv")
    df_csv1_train = pd.read_csv(f"CSV1_{reg}_train.csv")  # NEW 🔥

    keys = ['uj_lat','uj_lon','Z1','Z2']

    def rounded_view(df, cols=keys, nd=6):
        v = df[cols].astype(float).copy()
        for c in cols:
            v[c] = v[c].round(nd)
        return v

    rv_proj = rounded_view(df_proj)
    rv_csv1_test = rounded_view(df_csv1_test)
    rv_csv1_train = rounded_view(df_csv1_train)

    # ====== TEST subset ======
    df_proj_test_idx = rv_proj.merge(
        rv_csv1_test.drop_duplicates(),
        on=keys, how='inner'
    ).index
    df_proj_test = df_proj.iloc[df_proj_test_idx].copy()

    # ====== FRM TRAIN subset (Part 1 you requested) ======
    df_proj_train_FRM_idx = rv_proj.merge(
        rv_csv1_train.drop_duplicates(),
        on=keys, how='inner'
    ).index
    df_proj_train_FRM = df_proj.iloc[df_proj_train_FRM_idx].copy()

    # ====== FULL TRAIN = ALL - Test ======
    rv_test_keys = rounded_view(df_proj_test).drop_duplicates()
    anti = rv_proj.merge(rv_test_keys, on=keys, how='left', indicator=True)
    keep_mask = anti['_merge'].eq('left_only').to_numpy()
    df_proj_train = df_proj[keep_mask].copy()

    assert len(df_proj) == len(df_proj_train) + len(df_proj_test), \
        f"[{reg}] mismatch total={len(df_proj)}, train={len(df_proj_train)}, test={len(df_proj_test)}"

    # ====== SAVE ======
#     df_proj_train.to_csv(f"projection_matrix_{reg}_train.csv", index=False)
#     df_proj_test.to_csv(f"projection_matrix_{reg}_test.csv", index=False)
    df_proj_train_FRM.to_csv(f"projection_matrix_{reg}_train_FRM.csv", index=False)  # NEW 🔥

    print(f"✔ Saved: projection_matrix_{reg}_train.csv (N={len(df_proj_train)})")
    print(f"✔ Saved: projection_matrix_{reg}_test.csv  (N={len(df_proj_test)})")
    print(f"✔ Saved: projection_matrix_{reg}_train_FRM.csv (N={len(df_proj_train_FRM)})  <-- YOU WANT THIS")

print("\n✨ Part-1 FRM training subsets also saved! 🎯")



Processing region: CA
✔ Saved: projection_matrix_CA_train.csv (N=2773)
✔ Saved: projection_matrix_CA_test.csv  (N=22)
✔ Saved: projection_matrix_CA_train_FRM.csv (N=86)  <-- YOU WANT THIS

Processing region: NE
✔ Saved: projection_matrix_NE_train.csv (N=1950)
✔ Saved: projection_matrix_NE_test.csv  (N=19)
✔ Saved: projection_matrix_NE_train_FRM.csv (N=76)  <-- YOU WANT THIS

Processing region: SE
✔ Saved: projection_matrix_SE_train.csv (N=3560)
✔ Saved: projection_matrix_SE_test.csv  (N=16)
✔ Saved: projection_matrix_SE_train_FRM.csv (N=61)  <-- YOU WANT THIS

✨ Part-1 FRM training subsets also saved! 🎯


In [9]:
len(df_proj_train)

10873

In [10]:
len(df_proj_test)

97